Here I'm going to explore the notification moduel and how they work. First, I need to understand their fields

In [17]:
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
df = pd.read_json("../data/base_22012019.json")

In [5]:
notifications_df = df[df["model"] == "notifications.notification"]

In [7]:
notifications_df.shape

(13219, 3)

In [14]:
notifications_df["fields"].head(10)[34636]

{'meta': None,
 'task': 1,
 'user': 8,
 'level': 3,
 'viewed': False,
 'creation_date': '2018-10-28'}

Uma notificação possui os seguintes campos no log:
1. Meta 
2. User que coleta o id do user para qual a notificação foi enviada
3. Level 
4. Viewed que possui valor de verdadeiro ou falso indicando se a notifcação foi vista ou não pelo usuário
5. A data de criação daquela notificação

# Limpeza dos dados

é preciso extrair as colunas existentes dentro da coluna "fields", para isso, eu vou utilizar a função json_normalize, nativa do pandas.

In [77]:
def extract_flatten_dataframe(df, column, meta_list):
    df_fields = json_normalize(data=df[column], meta=meta_list)
    df_fields.index = df.index
    return df_fields.join(df, how="outer")

In [72]:
notification_fields = json_normalize(data=notifications_df["fields"], meta=["meta", "task", "user", "level", "viewed", "creation_date"] )
notification_fields.index = notifications_df.index
notification_df_flatted = notification_fields.join(notifications_df, how="outer")

In [76]:
notification_df_flatted.sample(10)

,creation_date,level,meta,task,user,viewed,fields,model,pk
40795,2018-12-01,3,None,16,92,False,"{'meta': None, 'task': 16, 'user': 92, 'level'...",notifications.notification,6302
42160,2018-12-07,4,None,8,53,False,"{'meta': None, 'task': 8, 'user': 53, 'level':...",notifications.notification,7727
35703,2018-11-09,3,None,2,8,False,"{'meta': None, 'task': 2, 'user': 8, 'level': ...",notifications.notification,1070
47437,2019-01-22,1,None,45,29,False,"{'meta': None, 'task': 45, 'user': 29, 'level'...",notifications.notification,13084
35820,2018-11-12,3,None,7,63,False,"{'meta': None, 'task': 7, 'user': 63, 'level':...",notifications.notification,1187
41702,2018-12-05,3,None,17,50,False,"{'meta': None, 'task': 17, 'user': 50, 'level'...",notifications.notification,7249
42703,2018-12-13,3,None,12,34,False,"{'meta': None, 'task': 12, 'user': 34, 'level'...",notifications.notification,8310
43584,2018-12-17,3,None,42,65,True,"{'meta': None, 'task': 42, 'user': 65, 'level'...",notifications.notification,9231
38472,2018-11-22,4,None,8,65,False,"{'meta': None, 'task': 8, 'user': 65, 'level':...",notifications.notification,3889
35289,2018-11-08,3,None,9,41,True,"{'meta': None, 'task': 9, 'user': 41, 'level':...",notifications.notification,656


In [38]:
notification_fields["meta"].value_counts()

2018-11-11T01:44:00Z    45
2018-11-14T12:33:00Z     6
2018-12-01T00:00:00Z     4
2018-11-27T22:34:00Z     3
2018-11-14T23:00:00Z     3
2018-11-12T13:33:00Z     3
2018-11-11T23:22:00Z     2
2018-11-13T15:19:00Z     1
2018-11-07T03:40:00Z     1
2018-12-02T02:26:00Z     1
2018-11-23T14:05:00Z     1
2018-11-19T02:30:00Z     1
2018-11-07T02:46:00Z     1
Name: meta, dtype: int64

In [23]:
notification_fields.dtypes

creation_date    object
level             int64
meta             object
task              int64
user              int64
viewed             bool
dtype: object

Como dá pra ver o pandas não consegue inferir muito bem os data types das colunas, vou utilizar meu conhecimento sobre elas para colocar valores mais baratos e que correspondem melhor aos seus valores.m

In [30]:
notification_fields = notification_fields.astype({"level": pd.CategoricalDtype()})

In [37]:
notification_fields["level"].describe()

count     13219
unique        4
top           3
freq       6554
Name: level, dtype: int64

In [31]:
notification_fields.dtypes

creation_date      object
level            category
meta               object
task                int64
user                int64
viewed               bool
dtype: object

In [20]:
notification_fields.shape

(13219, 6)

# Users Data
Como os usuários são o principal objeto de pesquisa deste relatório, eu preciso coletar os IDs deles para juntar as notificações e "dar à um responsável".

In [41]:
users_df = df[df["model"] == "users.user"]
users_df

,fields,model,pk
34528,{'password': 'pbkdf2_sha256$30000$5DROf4Pf3BPR...,users.user,1
34529,{'password': 'pbkdf2_sha256$30000$u3Lg5la328P5...,users.user,2
34530,{'password': 'pbkdf2_sha256$30000$Tg55KDswGl6t...,users.user,3
34531,{'password': 'pbkdf2_sha256$30000$sN16VkfYDIiS...,users.user,4
34532,{'password': 'pbkdf2_sha256$30000$x85db28ZviCz...,users.user,6
34533,{'password': 'pbkdf2_sha256$30000$YuhP7bGSlXwn...,users.user,7
34534,{'password': 'pbkdf2_sha256$30000$l7WAB2nBwfTg...,users.user,8
34535,{'password': 'pbkdf2_sha256$30000$DOj6RbuvrpDK...,users.user,9
34536,{'password': 'pbkdf2_sha256$30000$wbwdFEajjHed...,users.user,10
34537,{'password': 'pbkdf2_sha256$30000$DOPLvJQYYOz4...,users.user,11


In [44]:
users_df["fields"][34528]

{'password': 'pbkdf2_sha256$30000$5DROf4Pf3BPR$tPd1JbDlLqrUzoY/JcUSfparBrp7n0ERZKWgpfnbG/Y=',
 'last_login': '2019-01-22T12:36:36.447Z',
 'is_superuser': True,
 'email': 'admin@amadeus.br',
 'username': 'Administrador',
 'last_name': 'Geral',
 'social_name': None,
 'description': '',
 'image': '',
 'date_created': '2018-10-19T16:55:27.084Z',
 'last_update': '2018-11-04T03:31:13.036Z',
 'show_email': 1,
 'is_staff': True,
 'is_active': True,
 'groups': [],
 'user_permissions': []}

# Dicionário dos dados
1. Password: Senha criptografada
2. Last_Login: a última vez que o usuário logou no sistema
3. is_superuser: Significa que o usuário é um super usuário, tem privilégios ou admin.
4. email: e-mail pertecente ao usuário 
5. username: nome que o usuário quer que outros usuários o vejam
6. last_name: Sobrenome do usuário
7. social_name: Nome social que o usuário quer escolher para ser demonstrado 
8. description: ?
9. image: Caminho para a imagem do usuário
10. date_created: Data em que o usuário foi criado
11. last_update: a última vez que os dados do usuário foram modificados
12. show_email: Uma booleana que informa se o e-mail é visiível para outros usuários
13. is_staff: se ele é do tipo admin
14. is_active: Se o usuário está ativo, caso sim, ele pode entrar no sistema, senão, é impossível
15. groups: Grupos de permissão ao qual esse usuário pertence ("professor", "estudante"...)
16. user_permissions = permissões individuais que ele possui ("criar tópico" , "deletar tópico", "editar tópico")

In [45]:
# fields:  
user_fields = ["password", "last_login", "is_superuser", "email", "username", "last_name", "social_name", "description", 
               "imagem", "date_created", "last_update", "show_email", "is_staff", "is_active", "groups", "user_permissions"]

In [88]:
user_df_flatten = extract_flatten_dataframe(users_df, column="fields",meta_list=user_fields)

# Vou remover as seguintes colunas:
1. Groups, pois não possui nenhum valor diferente de vazio (ou nulo).

In [80]:
user_df_flatten["groups"].value_counts()

[]    106
Name: groups, dtype: int64

In [91]:
user_df_flatten_clean = user_df_flatten.drop(["groups", "fields", "password"], axis=1)

In [92]:
user_df_flatten_clean.sample(100)

,date_created,description,email,image,is_active,is_staff,is_superuser,last_login,last_name,last_update,show_email,social_name,user_permissions,username,model,pk
34554,2018-11-03T13:47:23.529Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,05200742544@r1.br,,True,False,False,2019-01-20T21:46:25.751Z,.,2018-11-03T13:47:23.564Z,1,IARA DE DEUS SILVA,[],IARA DE DEUS SILVA,users.user,28
34543,2018-11-03T13:41:29.174Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,10773458441@r1.br,,True,False,False,2019-01-21T18:07:01.743Z,.,2018-11-03T13:41:29.207Z,1,BRUNA LARISSA PEIXOTO DOS SANTOS,[],BRUNA LARISSA PEIXOTO DOS SANTOS,users.user,17
34582,2018-11-03T14:02:23.521Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,06238804580@r1.br,,True,False,False,2019-01-22T12:52:00.350Z,.,2018-11-03T14:02:23.555Z,1,DEISE SILVA DE OLIVEIRA,[],DEISE SILVA DE OLIVEIRA,users.user,56
34595,2018-11-03T14:09:12.499Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,86157567597@r1.br,,True,False,False,2019-01-21T18:01:36.044Z,.,2018-11-03T14:09:12.540Z,1,KAREN AKEMI OTSUKA,[],KAREN AKEMI OTSUKA,users.user,69
34630,2019-01-21T21:53:06.342Z,,igor.hh@outlook.com,users/20190120_113542.jpg,True,False,False,2019-01-21T21:54:36.406Z,Lima,2019-01-21T21:53:06.868Z,1,Igor,[],Igor,users.user,105
34596,2018-11-03T14:09:46.793Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,karinegsantos1@gmail.com,users/IMG_20180727_211040_057.jpg,True,False,False,2019-01-21T17:10:04.270Z,.,2018-11-09T14:23:27.136Z,1,Karine Gomes,[],KARINE GOMES SANTOS,users.user,70
34533,2018-10-27T18:03:57.447Z,<p>Usuário de testes.</p>,e2@amadeus.br,users/masculino_04_.jpg,True,False,False,2018-11-21T09:09:51.308Z,.,2018-11-17T20:09:57.884Z,1,Matheus Gabriel,[],Estudante 02,users.user,7
34546,2018-11-03T13:43:04.959Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,02536903583@r1.br,users/CR.jpg,True,False,False,2019-01-22T11:56:32.166Z,.,2018-11-11T22:33:44.486Z,1,CARLOS RAMON NUNES DA SILVA,[],CARLOS RAMON NUNES DA SILVA,users.user,20
34607,2018-11-03T14:15:18.260Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,09958033402@r1.br,users/IMG_20180712_230213_502.jpg,True,False,False,2019-01-21T19:33:56.981Z,.,2018-12-11T17:07:58.285Z,1,RANIELE CAVALCANTI MIRANDA,[],RANIELE CAVALCANTI MIRANDA,users.user,81
34557,2018-11-03T13:48:59.418Z,<p>\r\n\r\n\r\n\r\n\r\n<style>\r\n<!--table\r\...,09933013475@r1.br,,True,False,False,2019-01-21T17:12:37.188Z,.,2018-11-03T13:48:59.451Z,1,JORGE DE SOUSA RODRIGUES,[],JORGE DE SOUSA RODRIGUES,users.user,31


# Junção entre usuários e notificações
Para otimizar, irei coletar somente o username dos usuários durante o join com as notificações, depois eu irei buscar mais informações sobre eles.m